In [33]:
import os
import sys
import pandas as pd

In [34]:
from global_functions import up_load_shp, make_point, add_geo_info_shp,add_geo_info_gdb

In [35]:
#### תאריך
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [36]:
TAZ_V4_date='240404'

In [37]:
software_folder_location=r'W:\Data\Forecast\Tools\forecast_git\create_forecast_basic'

In [38]:
taz=up_load_shp(r'{}\background_files\TAZ_V4_{}.shp'.format(software_folder_location, TAZ_V4_date))

In [39]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'jeru_metro_jtmt_border_240402', 'jeru_metro')

In [40]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'in_jeru', 'in_jerusal')
taz.loc[taz['in_jerusal']==0,'in_jerusal']='no'

In [41]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'main_secto', 'main_secto')

In [42]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'SCHN_NAME', 'SCHN_NAME')

In [43]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'subdistrict2008','ENG_NAME_n')
taz['zonetype']=taz['ENG_NAME_n']

In [44]:

taz.main_secto.unique()

array(['U_Orthodox', 'arabs_behined_seperation_wall', 'Arab', 'Jewish',
       'Palestinian'], dtype=object)

In [45]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'muni_under_JTMT_ITM', 'Muni_Heb')

In [46]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'muni_under_JTMT_ITM', 'Sug_Muni')

In [47]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'muni_under_JTMT_ITM', 'CR_PNIM')

In [48]:
taz.loc[taz['main_secto']=='Palestinian','zonetype']='Palestinian'

In [49]:
taz.loc[taz['main_secto']=='Palestinian','Muni_Heb']=' '

In [50]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'highBusinessFlag', 'HighBiz')
taz.rename(columns={'HighBiz': 'highBusinessFlag'}, inplace=True)

In [51]:
taz=add_geo_info_shp(taz,taz,software_folder_location,'schDistrict','ID').rename(columns={'ID':'SCHOOLDISTRICT'})
taz.loc[taz['SCHOOLDISTRICT']==0,'SCHOOLDISTRICT']=999

In [52]:
taz=add_geo_info_shp(taz,taz,software_folder_location,'urban','Urban')

In [53]:
taz=add_geo_info_shp(taz,taz,software_folder_location,'EIProp','EIProp')

In [54]:
col=['poly_puma', 'F3', 'F2', 'F1']

for c in col:
    taz=add_geo_info_shp(taz,taz,software_folder_location,'poly_pumas',c)
    

In [55]:
col=['agri', 'Indus', 'Com_hotel', 'Business', 'Public']

for c in col:
    taz=add_geo_info_shp(taz,taz,software_folder_location,'emp_category_type',c)
    taz['{}'.format(c)]=taz['{}'.format(c)]/100
    

In [56]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'commuting_230712', 'commuting')
taz['commuting']=taz['commuting']/100

In [57]:
taz['ieold']=0
taz.loc[taz['jeru_metro']==1,'ieold']=taz['commuting']
taz['IEProp']=taz['ieold']

In [58]:
taz['perScaled']=1-taz['EIProp']

In [59]:
col_parking=['FreeBuffer',
'PaidBuffer',
'Rest_EmpBuffer',
'searchtime',
'walktime',
'cost']

for c in col_parking:
    taz=add_geo_info_gdb(taz,taz,software_folder_location,'parking_abm',c,c)

In [60]:
taz.loc[taz['Taz_num'] < 7001, 'Agg_taz_nu'] = taz['Taz_num'] // 100
taz.loc[taz['Taz_num'] >= 7001, 'Agg_taz_nu'] = taz['Taz_num'] // 10

In [61]:
col_with_fix_num=['area',
 'CITYCODE1',
 'CITYCODE2',
 'CITYCODE3',
 'CITYCODE4',
 'codeseq',
 'codeseqCons',
 'county',
 'majunivenr',
 'parktot',
 'superZone',
 'Taz1','slope','REGION']


In [62]:

taz[col_with_fix_num]=1

In [63]:
taz['yosh']=0

taz.loc[taz['zonetype']=='Judea and Samaria','yosh']=1

taz['jerusalem_city']=0

taz.loc[taz['in_jerusal']=='yes','jerusalem_city']=1

In [64]:
columns = taz.columns
# Create an empty DataFrame with only column names
empty_df = pd.DataFrame(columns=columns)

In [65]:
empty_df.to_excel(r"{}\background_files\col_names.xlsx".format(software_folder_location), index=False)

In [66]:
taz.to_file(r"{}\background_files\TAZ_V4_{}_with_geo_info.shp".format(software_folder_location, file_date), encoding='utf-8')

C:\Users\gidon\AppData\Local\Temp\ipykernel_24124\2075935453.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  taz.to_file(r"{}\background_files\TAZ_V4_{}_with_geo_info.shp".format(software_folder_location, file_date), encoding='utf-8')
